<a href="https://colab.research.google.com/github/fotrino/austral/blob/main/Gestion%20del%20Conocimiento/json_tp/json_ld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividades Web of Data Comisión A

## Integrantes:

* Del Villar, Javier
* Otrino, Facundo Damián
* Pistoya, Haydeé Soledad
* Rojas, Mariano Arturo
* Vaillard, Leandro Carlos

## Cargamos Librerías

In [41]:
import pandas as pd
import json
import itertools

## Cargamos JSONs

In [6]:
json_ecartelera = pd.read_json('https://raw.githubusercontent.com/fotrino/austral/main/Gestion%20del%20Conocimiento/json_tp/WW1984ecartelera.json', orient='index')
json_IMDB = pd.read_json('https://raw.githubusercontent.com/fotrino/austral/main/Gestion%20del%20Conocimiento/json_tp/WW1984IMDB.json', orient='index')
json_Tomatoes = pd.read_json('https://raw.githubusercontent.com/fotrino/austral/main/Gestion%20del%20Conocimiento/json_tp/WW1984Tomatoes.json', orient='index')

## Unificamos JSONs

In [7]:
df = pd.concat([json_ecartelera, json_IMDB, json_Tomatoes], axis=1)
df.columns = ['eCartelera', 'IMDB', 'RottenTomatoes']
df

eCartelera  ...                                     RottenTomatoes
@context                                           http://schema.org  ...                                  http://schema.org
@type                                                          Movie  ...                                              Movie
@id                https://www.ecartelera.com/peliculas/wonder-wo...  ...                                                NaN
name                                               Wonder Woman 1984  ...                                  Wonder Woman 1984
mainEntityOfPage   https://www.ecartelera.com/peliculas/wonder-wo...  ...                                                NaN
description        &#39;Wonder Woman 1984&#39; llegará a la panta...  ...                                                NaN
duration                                                     PT2H31M  ...                                                NaN
image              {'@type': 'ImageObject', 'url': 'https://img.e...  ...  https://resizing.flixster.com/sr2SNIPoCFaMLQt-...
countryOfOrigin                                               EE.UU.  ...                                                NaN
genre                                   [Acción, Aventura, Fantasía]  ...    [Action & Adventure, Science Fiction & Fantasy]
releasedEvent      {'@type': 'PublicationEvent', 'startDate': '20...  ...                                                NaN
productionCompany  {'@type': 'Organization', 'name': 'Warner Bros...  ...  {'@type': 'Organization', 'name': 'Warner Bros...
hasPart            {'@type': 'Clip', 'description': 'trailer', 't...  ...                                                NaN
director           [{'@type': 'Person', 'name': 'Patty Jenkins', ...  ...  [{'@type': 'Person', 'name': 'Patty Jenkins', ...
actor              [{'@type': 'Person', 'name': 'Gal Gadot', 'mai...  ...                                                NaN
aggregateRating    {'@type': 'AggregateRating', 'bestRating': '10...  ...  {'@type': 'AggregateRating', 'name': 'Tomatome...
url                                                              NaN  ...  https://www.rottentomatoes.com/m/wonder_woman_...
contentRating                                                    NaN  ...                                              PG-13
creator                                                          NaN  ...                                                NaN
trailer                                                          NaN  ...                                                NaN
datePublished                                                    NaN  ...                                               None
keywords                                                         NaN  ...                                                NaN
review                                                           NaN  ...  [{'@type': 'Review', 'reviewBody': 'Wonder Wom...
dateModified                                                     NaN  ...                          2021-06-10T21:02:19-07:00
dateCreated                                                      NaN  ...                          2017-09-13T08:24:15-07:00
actors                                                           NaN  ...  [{'@type': 'Person', 'name': 'Gal Gadot', 'sam...
character                                                        NaN  ...  [Diana Prince / Wonder Woman, Barbara Minerva ...
author                                                           NaN  ...  [{'@type': 'Person', 'name': 'Patty Jenkins', ...

[28 rows x 3 columns]

## Seleccionamos y combinamos valores de cada fuente de datos

In [8]:
json_total = pd.Series(dtype='float64')

In [61]:
enGenre = list(itertools.chain.from_iterable(
    [i.split(' & ') for i in df.loc['genre', 'RottenTomatoes']]
)) + df.loc['genre', 'IMDB']

enGenre = list(dict.fromkeys(enGenre))

enGenre

['Action', 'Adventure', 'Science Fiction', 'Fantasy']

In [66]:
json_total['@context'] = df.loc['@context', 'eCartelera']

json_total['@type'] = df.loc['@type', 'eCartelera']

json_total['name'] = df.loc['name', 'IMDB']

json_total['description'] = {
  "en": df.loc['description', 'IMDB'],
  "es":df.loc['description', 'eCartelera']
}

json_total['duration'] = df.loc['duration', 'IMDB']

json_total['image'] = df.loc['image', 'IMDB']

json_total['countryOfOrigin'] = df.loc['countryOfOrigin', 'eCartelera']

json_total['genre'] = {
  "en": enGenre,
  "es": df.loc['genre', 'eCartelera']
}

json_total['productionCompany'] = df.loc['productionCompany', 'RottenTomatoes']

json_total['director'] = df.loc['productionCompany', 'RottenTomatoes']

json_total['actor'] = df.loc['actors', 'RottenTomatoes']

json_total['aggregateRating'] = {
    "eCartelera": float(df.loc['aggregateRating', 'eCartelera']['ratingValue'].replace(',', '.')),
    "IMDB": df.loc['aggregateRating', 'IMDB']['ratingValue'],
    "RottenTomatoes": float(df.loc['aggregateRating', 'RottenTomatoes']['ratingValue']),
    "ownMetrics": (
    float(df.loc['aggregateRating', 'eCartelera']['ratingValue'].replace(',', '.')) + 
    df.loc['aggregateRating', 'IMDB']['ratingValue'] + 
    float(df.loc['aggregateRating', 'RottenTomatoes']['ratingValue']) / 10
    ) / 3
  }

json_total['url'] = [
    df.loc['@id', 'eCartelera'], 
    'https://www.imdb.com' + df.loc['url', 'IMDB'], 
    df.loc['url', 'RottenTomatoes']
    ]

json_total['contentRating'] = df.loc['contentRating', 'RottenTomatoes']

json_total['trailer'] = df.loc['hasPart', 'eCartelera']

json_total['datePublished'] = df.loc['releasedEvent', 'eCartelera']

json_total['keywords'] = df.loc['keywords', 'IMDB']

json_total['review'] = df.loc['review', 'RottenTomatoes']

json_total['character'] = df.loc['character', 'RottenTomatoes']

json_total['author'] = df.loc['creator', 'IMDB']

json_total

@context                                             http://schema.org
@type                                                            Movie
name                                                 Wonder Woman 1984
description          {'en': 'Diana must contend with a work colleag...
duration                                                       PT2H31M
image                https://m.media-amazon.com/images/M/MV5BYTlhNz...
countryOfOrigin                                                 EE.UU.
genre                {'en': ['Action', 'Adventure', 'Science Fictio...
productionCompany    {'@type': 'Organization', 'name': 'Warner Bros...
director             {'@type': 'Organization', 'name': 'Warner Bros...
actor                [{'@type': 'Person', 'name': 'Gal Gadot', 'sam...
aggregateRating      {'eCartelera': 7.6, 'IMDB': 5.4, 'RottenTomato...
url                  [https://www.ecartelera.com/peliculas/wonder-w...
contentRating                                                    PG-13
traile

## Exportamos el JSON totalizado

In [68]:
json_total.to_json('json_total.json')